# **Project 1: Finding Lane Lines on the Road** 


The goals / steps of this project are the following:

In this project we have to identify lane lines on the road using some of the helper functions provided with the project. We start with developing a pipeline on a series of individual images, and later apply the result to a video stream. Once we have this worked out, the next big part is to contribute to the **draw\_lines()** function and try to average and/or extrapolate the line segments we've detected to map out the full extent of the lane lines. The output of the project is an annotated version of the input video stream shown by 2 solid lines annotating the left and right lanes throughout most of the video

Final submissions of the project include:

* Code: Make a pipeline that finds lane lines on the road
* Write up: Reflect on your work in a written report

##### Pipeline description

My pipeline consisted of following steps:

* Read an image/frame from a video and get rid of noise and smoothen the image using Gaussian filter

* Convert from RGB to HSV color space to filter out into images with yellow lane (1st image) and white lane (2nd image) markings and combine both (3rd image)

* Apply Canny on the combined image to find all edges (1st image) and then filter the only ones within the ROI (2nd image)

* Apply Hough transform to find lines in the image (1st image) and then combine (2nd image) the found lines with the originial image frame

Hough transform is applied using **hough\_lines()** function which calls **draw\_lines()** function to draw lines on the image frames. A good portion of the project is dedicated to modifying the **draw\_lines()** function to annotate the resulting video with 2 solid lines on top of left and right lanes.


## Import Packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

## Helper Functions

In [2]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

## Improving the draw_lines() function

**At this point, if you were successful with making the pipeline and tuning parameters, you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform. As mentioned previously, try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines. You can see an example of the result you're going for in the video "P1_example.mp4".**

**The modified draw_lines function should draw a single, solid line over the left lane line and a single, solid line over the right lane line. The lines should start from the bottom of the image and extend out to the top of the region of interest.**

In [3]:
def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    pos_m = np.array([],dtype=np.int32).reshape(0,1)
    neg_m = np.array([],dtype=np.int32).reshape(0,1)
    
    pos_c = np.array([],dtype=np.int32).reshape(0,2)
    neg_c = np.array([],dtype=np.int32).reshape(0,2)
    
    pos_m_lines = np.array([],dtype=np.int32).reshape(0,4)
    neg_m_lines = np.array([],dtype=np.int32).reshape(0,4)
    
    ##### CAPTURING SLOPES AND CENTERS OF LINES WITH REASONABLE SLOPE VALUES #####
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            m = (y2-y1)/(x2-x1)
            c = np.array([np.int32((x1+x2)/2),np.int32((y1+y2)/2)])
            
            m_lim1 = 0.5
            m_lim2 = 0.8
            
            # Filter out lines based on slopes and get rid of the lines with extreme slopes
            if m < -m_lim1 and m > -m_lim2: # Since in an image compared to 2D cartesian co-ordinate system y is increasing downwards
                # collect all pos slope values
                pos_m = np.vstack([pos_m,m])                
                # collect the centers of these points too
                pos_c = np.vstack([pos_c,c])
            elif m > m_lim1 and m < m_lim2:
                # collect all neg slope values
                neg_m = np.vstack([neg_m,m])
                # collect all neg slope points
                neg_c = np.vstack([neg_c,c])
    
    ###### FINDING AVERAGE SLOPE & INTERCEPT FOR EACH LANE FOR EXTRAPOLATION #####
    
    # Format it into an array so that the 2 for loops below can iterate over its elements 
    pos_c = [pos_c]
    neg_c = [neg_c]
    
    # Using slope intercept form (y-y')=m(x-x') to find out 2 points to draw the l & r lines
    # x,y we calculate as the average of the center points, set y'= img.height/2 then using
    # avg slope m, calculate x'. That becomes one point. For the next one set y'= img.height
    # and calculate x'. This becomes the second point. Draw a line using both. Repeat for the
    # other lane
        
    # Find avg of l & r slope values for smoothing lines frame-to frame if there are lines 
    # with valid slopes in a frame else provide average slope between the limits
        
    if len(pos_m) != 0:
        l_m_avg = sum(pos_m)/len(pos_m)        
    else:
        l_m_avg = -(m_lim1 + m_lim2)/2
        
    if len(neg_m) != 0:
        r_m_avg = sum(neg_m)/len(neg_m)
    else:
        r_m_avg = (m_lim1 + m_lim2)/2
    
    #print("L slopes avg",l_m_avg)
        
    imshape = img.shape
    
    # Finding top intercepts for LEFT lane
    l_p1_y_prime = int(imshape[0]/2) + 100 # more than half the height of img to align well with the lane markings
    l_p1_x_prime_list = []
    
    for point in pos_c:
        for x,y in point:            
            x_prime = x - int((y-l_p1_y_prime)/l_m_avg)  
            
            # limits applied above to filter top intercepts
            if x_prime > int(0.4*imshape[1]) and x_prime < int(0.5*imshape[1]):
                l_p1_x_prime_list = l_p1_x_prime_list + [x_prime]           
    
    # Averaging all the intercepts for smoothing    
    if len(l_p1_x_prime_list) != 0:
        l_p1_x_prime = int(sum(l_p1_x_prime_list)/len(l_p1_x_prime_list))
    else:
        l_p1_x_prime = int(0.45*imshape[1]) # avg of the limits applied above to filter top intercepts        
        
    # Finding bottom intercepts for LEFT lane   
    l_p2_y_prime = imshape[0] # height of image
    l_p2_x_prime_list = []
    
    for point in pos_c:
        for x,y in point:
            x_prime = x - int((y-l_p2_y_prime)/l_m_avg)
            
            # limits applied above to filter bottom intercepts
            if x_prime > int(0.10*imshape[1]) and x_prime < int(0.30*imshape[1]):
                l_p2_x_prime_list = l_p2_x_prime_list + [x_prime]
       
    # Averaging all the intercepts for smoothing
    if len(l_p2_x_prime_list) != 0:
        l_p2_x_prime = int(sum(l_p2_x_prime_list)/len(l_p2_x_prime_list))
    else:
        l_p2_x_prime = int(0.20*imshape[1])        
        
    pos_line = np.array([l_p1_x_prime,l_p1_y_prime,l_p2_x_prime,l_p2_y_prime])
    
    # Finding top intercepts for RIGHT lane
    r_p1_y_prime = int(imshape[0]/2) + 100 # more than half the height of img to align well with the lane markings
    r_p1_x_prime_list = []
    
    for point in neg_c:
        for x,y in point:
            x_prime = x - int((y-r_p1_y_prime)/r_m_avg)            
            # limits applied above to filter top intercepts
            if x_prime > int(0.55*imshape[1]) and x_prime < int(0.65*imshape[1]):
                r_p1_x_prime_list = r_p1_x_prime_list + [x_prime]
    
    # Averaging all the intercepts for smoothing
    if len(r_p1_x_prime_list) != 0:
        r_p1_x_prime = int(sum(r_p1_x_prime_list)/len(r_p1_x_prime_list))
    else:
        r_p1_x_prime = int(0.60*imshape[1])        
        
    # Finding bottom intercept for RIGHT lane
    r_p2_y_prime = imshape[0] # height of image
    r_p2_x_prime_list = []
    
    for point in neg_c:
        for x,y in point:
            x_prime = x - int((y-r_p2_y_prime)/r_m_avg)
            
            # limits applied above to filter bottom intercepts
            if x_prime > int(0.75*imshape[1]) and x_prime < int(1.0*imshape[1]):
                r_p2_x_prime_list = r_p2_x_prime_list + [x_prime]
    
    # Averaging all the intercepts for smoothing
    if len(r_p2_x_prime_list) != 0:
        r_p2_x_prime = int(sum(r_p2_x_prime_list)/len(r_p2_x_prime_list))
    else:
        r_p2_x_prime = int(0.85*imshape[1])        
        
    neg_line = np.array([r_p1_x_prime,r_p1_y_prime,r_p2_x_prime,r_p2_y_prime])
    
    return pos_line, neg_line

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)    
   
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)    
    pos_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    neg_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    
    [pos,neg] = draw_lines(line_img, lines)
    
    final_pos_line = [pos]
    final_neg_line = [neg]
    
    for x1,y1,x2,y2 in final_pos_line:
        cv2.line(pos_img, (x1, y1), (x2, y2), color=[0, 255, 255], thickness=10)
    
    for x1,y1,x2,y2 in final_neg_line:
        cv2.line(neg_img, (x1, y1), (x2, y2), color=[0, 255, 0], thickness=10)
    
    line_img = pos_img + neg_img
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test Images

Build your pipeline to work on the images in the directory "test_images"  

In [4]:
#reading in an image
import os
files = os.listdir("test_images/")

## Lane Finding Pipeline


In [5]:
for file in files:
    if file[0:3] != 'out':
   
        image = mpimg.imread('test_images/'+file)
        
        # Apply Gaussian filter to smoothen the image frame
        blur_img = gaussian_blur(image,3)
        
        # Convert to HSV color space
        hsv = cv2.cvtColor(blur_img,cv2.COLOR_BGR2HSV)
        
        # Filter out yellow and white lanes        
        Y_LOWER = np.array([2,2,210])
        Y_UPPER = np.array([255,255,255])
        Y_img = cv2.inRange(hsv,Y_LOWER, Y_UPPER)
    
        W_LOWER = np.array([3,1,140])
        W_UPPER = np.array([255,30,255])
        W_img = cv2.inRange(hsv,W_LOWER, W_UPPER)
        
        # Combine yellow and white filtered images
        color_filtered_image = Y_img + W_img
        
        # Apply Canny
        low_threshold = 75
        high_threshold = 225
        canny_img = canny(color_filtered_image,low_threshold, high_threshold)
        
        # Defining ROI
        imshape = image.shape        
        vertices = np.array([[(0,imshape[0]),(460, 320), (500, 320), (imshape[1],imshape[0])]], dtype=np.int32)
        p0 = vertices[0][0]
        p1 = vertices[0][1]
        p2 = vertices[0][2]
        p3 = vertices[0][3]

        # Draw the ROI on image for reference
        line = [[np.append(p0,[p1])],[np.append(p1,[p2])],[np.append(p2,[p3])],[np.append(p3,[p0])]]
        #draw_lines(image,line, color=[255,255,0],thickness = 3)
        canny_img_with_ROI = region_of_interest(canny_img,vertices)         

        # Defining Hough transform parameters
        rho = 1 # distance resolution in pixels of the Hough grid
        theta = np.pi/180 # angular resolution in radians of the Hough grid
        threshold = 20     # minimum number of votes (intersections in Hough grid cell)
        min_line_len = 5 #minimum number of pixels making up a line
        max_line_gap = 3    # maximum gap in pixels between connectable line segments

        # Run Hough on extracted ROI
        line_image = hough_lines(canny_img_with_ROI, rho, theta, threshold, min_line_len, max_line_gap)        
        final_image = weighted_img(line_image,image)        
       
        # Save the outputs
        mpimg.imsave('test_images/out_'+file,final_image)
        

# Test on Videos now

In [6]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

# Lane Finding Pipeline here for videos

In [7]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    
    # Apply Gaussian filter to smoothen the image frame
    blur_img = gaussian_blur(image,3)

    # Convert to HSV color space
    hsv = cv2.cvtColor(blur_img,cv2.COLOR_BGR2HSV)
    
    # Filter out yellow and white lanes
    Y_LOWER = np.array([2,2,210])
    Y_UPPER = np.array([255,255,255])
    Y_img = cv2.inRange(hsv,Y_LOWER, Y_UPPER)
    
    W_LOWER = np.array([3,1,140])
    W_UPPER = np.array([255,30,255])
    W_img = cv2.inRange(hsv,W_LOWER, W_UPPER)
    
    # Combine yellow and white filtered images
    color_filtered_image = Y_img + W_img
    
    # Apply Canny
    low_threshold = 75
    high_threshold = 225
    canny_img = canny(color_filtered_image,low_threshold, high_threshold)

    # Defining ROI
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(460, 320), (500, 320), (imshape[1],imshape[0])]], dtype=np.int32)
    p0 = vertices[0][0]
    p1 = vertices[0][1]
    p2 = vertices[0][2]
    p3 = vertices[0][3]

    # Draw the ROI on image for reference 
    line = [[np.append(p0,[p1])],[np.append(p1,[p2])],[np.append(p2,[p3])],[np.append(p3,[p0])]]
    #draw_lines(image,line, color=[255,255,0],thickness = 3)
    canny_img_with_ROI = region_of_interest(canny_img,vertices)         

    # Define Hough transform parameters
    rho = 1 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 20     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 5 #minimum number of pixels making up a line
    max_line_gap = 3    # maximum gap in pixels between connectable line segments

    # Run Hough on extracted ROI
    line_image = hough_lines(canny_img_with_ROI, rho, theta, threshold, min_line_len, max_line_gap)    
    result = weighted_img(line_image,image)
    return result

# Save & Display Output videos

In [8]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████▉| 681/682 [00:20<00:00, 32.43it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

Wall time: 21.8 s


In [9]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [10]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

[MoviePy] >>>> Building video extra.mp4
[MoviePy] Writing video extra.mp4


100%|████████████████████████████████████████| 251/251 [00:14<00:00, 17.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: extra.mp4 

Wall time: 16.3 s


In [11]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))